In [167]:
!pip install rutermextract

import pandas as pd
import openpyxl
from rutermextract import TermExtractor

Используем библиотеку openpyxl для чтения xlsx файлов, библиотека pandas для работы с данными и библиотека rutermextract для получения ключевых слов (тегов)

In [276]:
wb_obj = openpyxl.load_workbook("dataset.xlsx")
sheet = wb_obj.active

df = pd.DataFrame(columns=["Название компетенции", "Раздел", "Важность раздела", "Навыки", "Теги"])
df = df.astype({"Теги": 'object'})

Составляем таблицу, состоящую из названия компетенции, раздела, важности этого раздела в %, навыков и тегов компетенции.

In [277]:
i = 1
for row in sheet.iter_rows():
    df_row = {}
    tags = []
    df_row["Название компетенции"] = sheet[f"A{i}"].value #Получаем название компетенции из столбца А
    for tag in term_extractor(sheet[f"B{i}"].value)[:15]:
      tags.append(tag.normalized) #Получаем 15 первых тегов, библиотека сортирует их по важности, поэтому мы получаем 15 самых ключевых
    df_row['Теги'] = '; '.join(tags) #Переводим их в читаемый текст, разделяя
    for j in range(2, len(row) - 2, 3): #Идём по строкам, считывая разделы, их важность и навыки
      if row[j].value == None:
        continue
      if row[j].value == 'Всего':
        continue
      #Данные очень "грязные", условиями отсекаю лишние разделы
      df_row["Раздел"] = row[j].value
      #В некоторых местах попадаются данные с текстом, пробуем его таким образом убрать
      if type(row[j+1].value) != int and type(row[j+1].value) != float:
        try:
          df_row["Важность раздела"] = float(row[j + 1].value[:-2])
        except (ValueError, TypeError):
          continue
      else:
        df_row["Важность раздела"] = float(row[j + 1].value)
      df_row["Навыки"] = row[j + 2].value
      #Добавляем данные из словаря в датафрейм
      df = df.append(df_row, ignore_index=True)
    i += 1

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Заполняем датафрейм данными, теги получаем из описания компетенции используя библиотеку rutermextract, не добавляем описание компетенции в датафрейм т.к. при обучении модели оно нам не понадобится — мы уже извлекли из него ключевые слова и обучаться будем на них

In [278]:
data = df.groupby(["Название компетенции"]).last().sort_values(by="Важность раздела", ascending=False)
data.head(10)

,Раздел,Важность раздела,Навыки,Теги
Название компетенции,,,,
Ветеринария,Решение производственных (ситуационных) задач,72.0,Специалист должен знать и понимать: • систему ...,ветеринарный специалист; животные; сырьё живот...
Ветеринария (Юниоры),Решение производственных (ситуационных) задач,72.0,Специалист должен знать и понимать: • систему ...,ветеринарный специалист; животные; сырьё живот...
Облицовка плиткой (16-22),Закрепление,60.0,Специалист должен знать и понимать:  Ряд мето...,плиточник; работа; крупные проекты; высочайший...
Облицовка плиткой (юниоры),Закрепление,60.0,Специалист должен знать и понимать:  Ряд мето...,плиточник; работа; крупные проекты; высочайший...
Звукорежиссура (Юниоры),Технические аспекты и общие характеристики,50.0,Специалист должен знать и понимать:  Технолог...,запись; эфир; редакция; работа; разные вариант...
Лазерные технологии (Lasertechnology),Эксплуатация лазерного оборудования,50.0,Специалист должен знать и понимать: • этапы на...,лазерные технологии; лазерные системы; автомат...
Звукорежиссура,Технические аспекты и общие характеристики,50.0,Специалист должен знать и понимать:  Технолог...,запись; эфир; редакция; работа; разные вариант...
Технологии моды,"Технологии раскроя, шитья, отделки",45.0,Специалист должен знать и понимать: • Важность...,специалист; технологии моды; сфера; одежда; ма...
Лабораторный химический анализ,"Обработка, анализ и оформление полученных резу...",45.0,Специалист должен знать и понимать:  Правила ...,химическое анализ; лаборанты; химические лабор...


Как мы можем тут видеть, возможно сгруппировать данные по необходимой компетенции и вывести её разделы, отсортировать по важности раздела

In [279]:
df[df['Раздел'].str.contains("Элек")]

,Название компетенции,Раздел,Важность раздела,Навыки,Теги
177,Водные технологии,Электрика,10.0,"Информация, знание и понимание которой требует...",сточные воды; очистка; окружающая среда; техно...
229,Командная работа на производстве,Электроника,10.0,Специалист должен знать и понимать: ❖ принци...,компетенция; способность; специалисты; сборка;...
608,Ремонт и обслуживание легковых автомобилей,"Электрические и механические системы, их взаим...",25.0,Специалист должен знать и разбираться: • в сис...,легковые автомобили; ремонт; обслуживание; мас...
613,Ремонт и обслуживание легковых автомобилей (Юн...,"Электрические и механические системы, их взаим...",25.0,Специалист должен знать и разбираться: • в сис...,легковые автомобили; ремонт; обслуживание; мас...


Здесь мы можем получить схожие разделы в различных компетенциях не зависимо от их важности

In [280]:
df.to_csv("dataset.csv", index=None)

Сохраняем наши данные в формат csv